In [1]:
!pip install gradio
!pip install torchvision

     --------------------------------------- 18.1/18.1 MB 25.1 MB/s eta 0:00:00
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
     ---------------------------------------- 45.1/45.1 kB ? eta 0:00:00
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
     ---------------------------------------- 318.7/318.7 kB ? eta 0:00:00
     ------------------------------------- 443.6/443.6 kB 28.9 MB/s eta 0:00:00
  Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
     ---------------------------------------- 62.5/62.5 kB ? eta 0:00:00
     ---------------------------------------- 73.5/73.5 kB ? eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 28.5 MB/s eta 0:00:00
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
     ------------------------------------ 133.6/133.6 kB 986.9 kB/s eta 0:00:00
     -------------------------------------- 95.2/95.2 kB 679.4 kB/s eta 0:00:00
  Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
  Us

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.18.1 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.20 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.
botocore 1.27.28 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.4.0 which is incompatible.


     ---------------------------------------- 1.6/1.6 MB 9.9 MB/s eta 0:00:00
     -----------------                      95.0/204.1 MB 16.4 MB/s eta 0:00:07


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transformation
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load training and test dataset
train_dataset = torchvision.datasets.GTSRB(
    root='./data', split='train', download=True, transform=transform)
test_dataset = torchvision.datasets.GTSRB(
    root='./data', split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# List of class labels
# Manual class names
classes = [
    "Speed limit (20km/h)", "Speed limit (30km/h)", "Speed limit (50km/h)",
    "Speed limit (60km/h)", "Speed limit (70km/h)", "Speed limit (80km/h)",
    "End of speed limit (80km/h)", "Speed limit (100km/h)", "Speed limit (120km/h)",
    "No passing", "No passing for vehicles over 3.5 metric tons",
    "Right-of-way at the next intersection", "Priority road", "Yield", "Stop",
    "No vehicles", "Vehicles over 3.5 metric tons prohibited", "No entry",
    "General caution", "Dangerous curve to the left", "Dangerous curve to the right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right",
    "Road work", "Traffic signals", "Pedestrians", "Children crossing",
    "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing",
    "End of all speed and passing limits", "Turn right ahead", "Turn left ahead",
    "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout mandatory", "End of no passing",
    "End of no passing by vehicles over 3.5 metric tons"
]



ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TrafficSignNet(nn.Module):
    def __init__(self):
        super(TrafficSignNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 43)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 → 16x16
        x = self.pool(F.relu(self.conv2(x)))  # 16x16 → 8x8
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = TrafficSignNet()


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


Epoch 1, Loss: 1.2947314344435858
Epoch 2, Loss: 0.19432496428668355
Epoch 3, Loss: 0.09118090882982245
Epoch 4, Loss: 0.05706507968381315
Epoch 5, Loss: 0.03676250735208464


In [ ]:
torch.save(model.state_dict(), "gtsrb_model.pth")


In [ ]:
import gradio as gr
import torch
from PIL import Image
import torch.nn.functional as F

# Load the trained model
model = TrafficSignNet()
model.load_state_dict(torch.load("gtsrb_model.pth", map_location=torch.device("cpu")))
model.eval()

# Predict function with sectional output
def predict(img: Image.Image):
    image = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)

        class_name = classes[predicted.item()]
        precaution = precautions.get(class_name, "Drive safely.")
        confidence_pct = confidence.item() * 100

        return f"""
<div class='result-section'>
  <div class='box detected'>
    <h3>🛑 Detected Sign</h3>
    <p>{class_name}</p>
  </div>
  <div class='box confidence'>
    <h3>🔒 Confidence</h3>
    <p>{confidence_pct:.2f}%</p>
  </div>
  <div class='box precaution'>
    <h3>⚠️ Precaution</h3>
    <p>{precaution}</p>
  </div>
</div>
        """

# Gradio UI with blue theme and sections
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    <div style="text-align:center; padding: 20px;">
        <h1 style="font-size: 2.8em; color: #01ffff;">🚦 Traffic Sign Recognition System</h1>
        <p style="font-size: 1.2em; color: #1e3799;">
            Upload a traffic sign image to detect the sign and receive real-time precaution tips.
        </p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(label="📷 Upload Traffic Sign", type="pil")
            submit_btn = gr.Button("🔍 Predict", elem_classes="custom-button")
        with gr.Column(scale=2):
            output_box = gr.HTML(label="Prediction Result")

    submit_btn.click(fn=predict, inputs=image_input, outputs=output_box)

    # Custom CSS for styling sections
    gr.HTML("""
    <style>
        .custom-button {
            background: linear-gradient(135deg, #3498db, #2ecc71);
            color: white !important;
            border: none !important;
            border-radius: 10px;
            font-size: 1em;
            padding: 12px 24px;
            transition: 0.3s ease;
        }

        .custom-button:hover {
            background: linear-gradient(135deg, #2980b9, #27ae60);
        }

        .result-section {
            font-family: 'Segoe UI', sans-serif;
            color: #01FFF;
        }

        .box {
            background: #01FFF;
            border: 2px solid #3498db;
            border-radius: 10px;
            padding: 15px 20px;
            margin: 10px 0;
        }

        .box h3 {
            margin: 0 0 5px;
            color: #0a3d62;
        }

        .box p {
            font-size: 1.1em;
            margin: 0;
        }
    </style>
    """)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://17a3814621aa53e022.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
